In [ ]:
import os, sys, pickle

import numpy as np
import pandas as pd

from scipy.optimize import curve_fit

import matplotlib.pyplot as plt

# Load data

In [ ]:
file = 'INSERT DIRECTORY' # Directory to data generated by BJ_filter.ipynb

all_df = pd.read_pickle(file)

In [ ]:
# Selection of Passed BJs or not.
df = all_df.loc[(all_df.trial != 'trial1') & (all_df.trial != 'trial2')]
clean = True
if clean:
    df = df.loc[df.passed == 1]

In [ ]:
logGs = np.vstack(df.logG.values)
Zs = np.vstack(df.Z.values)
plats = df.PlateauLen.values

# Constants

In [ ]:
def gauss(x, A, mu, sig):
    y = A * np.exp((-1 * (x - mu) ** 2) / (2 * sig**2))
    return y

In [ ]:
logG_bins = 128
z_bins = 106

logG_max, logG_min = 0.8, -6
z_max, z_min = 5E-3, -0.3E-3

logG_max_counts = 90000
logGz_max_counts = 4000

# 1D Histograms

In [ ]:
bin_edges = np.histogram_bin_edges(logGs.flatten(), bins=logG_bins)
bin_mids = bin_edges[:-1] + (np.diff(bin_edges) / 2)

counts, _ = np.histogram(logGs.flatten(), bins=bin_edges)

fit_lims = (-3.5, -2.3)
idxs = (bin_mids > fit_lims[0]) & (bin_mids < fit_lims[1])
x = bin_mids[idxs]
y = counts[idxs]

params, cov = curve_fit(gauss, x, y)

fig, ax = plt.subplots()
ax.bar(bin_mids, counts, width=np.diff(bin_edges))
ax.plot(bin_mids, gauss(bin_mids, *params), color='red')

ax.set(ylim=(0, logG_max_counts), xlim=(logG_min, logG_max))
print(params)

In [ ]:
fig, ax = plt.subplots()
ax.barh(bin_mids, counts, height=np.diff(bin_edges))

ax.set(xlim=(0, logG_max_counts), ylim=(logG_min, logG_max))

# 2D Histogram

In [ ]:
fig, ax = plt.subplots()
# h = ax.hist2d(Zs.flatten(), logGs.flatten(), bins=(z_bins, logG_bins), cmin=0, cmax=logGz_max_counts, range=((z_min, z_max), (logG_min, logG_max)), cmap='Blues')
h = ax.hist2d(Zs.flatten(), logGs.flatten(), bins=(z_bins, logG_bins), cmin=0, cmax=logGz_max_counts, range=((z_min, z_max), (logG_min, logG_max)))


## Plateau Length

In [ ]:
bin_edges = np.linspace(z_min, z_max, z_bins+1)
bin_mids = bin_edges[:-1] + (np.diff(bin_edges) / 2)

counts, _ = np.histogram(plats[~pd.isnull(plats)], bins=bin_edges)

params, cov = curve_fit(gauss, bin_mids, counts)

fig, ax = plt.subplots()

ax.bar(bin_mids, counts, width=np.diff(bin_edges))
ax.plot(bin_mids, gauss(bin_mids, *params), color='red')
ax.set(xlim=(z_min, z_max))

# Figure

In [ ]:
# Plot a combined figure of all three histogram types
fig, axs = plt.subplots(2, 2, figsize=(3, 3), dpi=600, gridspec_kw={'width_ratios':[2, 1], 'height_ratios':[2, 1]})
axs[1, 1].remove()

norm = plt.Normalize(0, logGz_max_counts)
# h1 = axs[0, 0].hist2d(Zs.flatten(), logGs.flatten(), bins=(z_bins, logG_bins), cmin=0, cmax=logGz_max_counts, range=((z_min, z_max), (logG_min, logG_max)), cmap='Blues')
h1 = axs[0, 0].hist2d(Zs.flatten(), logGs.flatten(), bins=(z_bins, logG_bins), range=((z_min, z_max), (logG_min, logG_max)), cmap='Blues', norm=norm)

bin_edges = np.histogram_bin_edges(logGs.flatten(), bins=logG_bins)
bin_mids = bin_edges[:-1] + (np.diff(bin_edges) / 2)
counts, _ = np.histogram(logGs.flatten(), bins=bin_edges)
axs[0, 1].barh(bin_mids, counts, height=np.diff(bin_edges))
axs[0, 1].set(xlim=(0, logG_max_counts), ylim=(logG_min, logG_max))

bin_edges = np.linspace(z_min, z_max, z_bins+1)
bin_mids = bin_edges[:-1] + (np.diff(bin_edges) / 2)
counts, _ = np.histogram(plats[~pd.isnull(plats)], bins=bin_edges)
axs[1, 0].bar(bin_mids, counts, width=np.diff(bin_edges))
axs[1, 0].set(xlim=(z_min, z_max))

axs[0, 0].set(xticklabels=[])
axs[0, 1].set(xticklabels=[], yticklabels=[], xticks=[])
axs[1, 0].set(yticklabels=[], yticks=[])

axs[0, 0].set_ylabel('log(G/G0)', weight='bold')
axs[1, 0].set_ylabel('Counts', weight='bold')
axs[1, 0].set_xlabel('Displacement /' + chr(956) +  'm', weight='bold')
axs[0, 1].set_xlabel('Counts', weight='bold')


sm = plt.cm.ScalarMappable(cmap='Blues', norm=norm)
sm.set_array([])

cax = axs[0, 0].inset_axes([0.75, 0.48, 0.05, 0.47])
cbar = plt.colorbar(sm, ax=axs[0, 0], cax=cax)
cax.tick_params(axis='y', labelsize=6, right=False, length=-2)#, color='White', )#, labelcolor='White')

axs[0, 0].tick_params(width=1.5)
axs[0, 1].tick_params(width=1.5)
axs[1, 0].tick_params(width=1.5)
for axis in ['top', 'right', 'bottom', 'left']:
    axs[0, 0].spines[axis].set_linewidth(1.5)
    axs[0, 1].spines[axis].set_linewidth(1.5)
    axs[1, 0].spines[axis].set_linewidth(1.5)
    
axs[0, 0].annotate('(a)', xy=(-0.35, 1.05), xytext=(0, 0), xycoords='axes fraction', textcoords='offset pixels', weight='bold')
axs[0, 1].annotate('(b)', xy=(-0.3, 1.05), xytext=(0, 0), xycoords='axes fraction', textcoords='offset pixels', weight='bold')
axs[1, 0].annotate('(c)', xy=(-0.35, 1.05), xytext=(0, 0), xycoords='axes fraction', textcoords='offset pixels', weight='bold')
    
# plt.subplots_adjust(left=-0.2, right=1.2, bottom=0, top=1)

# fig.savefig("Rot2 Clean.png", bbox_inches='tight')